In [ ]:
class A:
    def __init__(self, x):
        self.x = x
    def f(self):
        print(f'{self.x} in A')

class B(A):
    def __init__(self, x, y):
        super(B, self).__init__(x)
        self.y = y
    def f(self):
        print(f'{self.x} in B')
        super(B, self).f()

In [ ]:
b = B(1, 2)
b.f()

In [ ]:
class C(B):
    def f(self):
        super(self.__class__.__bases__[0], self).f()
        print(f'{self.x} in C')

In [ ]:
c = C(1, 2)
c.f()
# c.__class__.__bases__[0].

In [ ]:
from ultralytics import YOLO

In [ ]:
TRAIN = False

In [ ]:
if TRAIN:
    # load model
    model = YOLO("yolov8n.yaml").load("yolov8n.pt")

In [ ]:
if TRAIN:
    # train model
    train_result = model.train(data='coco128.yaml', epochs=100, imgsz=640, device='cuda:0')

In [ ]:
### TESTING YOLO.load() ###
trained_model_paths = [
    "C:/repos/edge-node/core/xperimental/lpd/runs/detect/train15/weights/best.pt",
    "C:/repos/edge-node/core/xperimental/lpd/runs/detect/train13/weights/best.pt"
]

In [ ]:
# IMPORTS
from copy import deepcopy
import torch as th
import os
import cv2

In [ ]:
# Environment variables
os.environ['KMP_DUPLICATE_LIB_OK'] = 'true'

In [ ]:
def compare_models(m1, m2):
  m1_state, m2_state = m1.state_dict(), m2.state_dict()
  n_found, n_diff = 0, 0

  for key, value in m1_state.items():
    if key in m2_state.keys():
      if not (m2_state[key] == value).all():
        # print(f'  {key} is different')
        n_diff += 1
    else:
      # print(f'  {key} not found in {model_paths[j]}')
      n_found += 1
    # endif key in m2_state.keys()
  # endfor key, value in m1_state.items()
  for key, value in m2_state.items():
    if key in m1_state.keys():
      if not (m1_state[key] == value).all():
        # print(f'  {key} is different')
        n_diff += 1
    else:
      # print(f'  {key} not found in {model_paths[i]}')
      n_found += 1
    # endif key in m1_state.keys()
  # endfor key, value in m2_state.items()
  print(f'  {n_found} keys not found, {n_diff} keys are different')
  print(f'{"not " if n_diff + n_found > 0 else ""}matching')
  return (n_diff + n_found) == 0

In [ ]:
###TESTING METHODS###
test_images = {
  'train': [
    "_local_cache/_data/Dataset_LPLR_v5.2.2_for_yolo-no_diff/images/1.Train/Cars_with_LP/Complexity_2/9.OTHER/000014_car 0.70_0.jpg",
    "_local_cache/_data/Dataset_LPLR_v5.2.2_for_yolo-no_diff/images/1.Train/Cars_with_LP/Complexity_2/9.OTHER/000122_car 0.94_2.jpg"
  ],
  'dev': [
    "_local_cache/_data/Dataset_LPLR_v5.2.2_for_yolo/images/2.Dev/Cars_with_LP/Complexity_2/4.GTS/dataset_builder_1__DATASET_BUILDER_01__DATASET_BUILDER_603.jpg",
    "_local_cache/_data/Dataset_LPLR_v5.2.2_for_yolo-no_diff/images/2.Dev/Cars_with_LP/Complexity_3/4.GTS/dataset_builder_2__DATASET_BUILDER_01__DATASET_BUILDER_28.jpg",
    "_local_cache/_data/Dataset_LPLR_v5.2.2_for_yolo-no_diff/images/2.Dev/Cars_without_LP/Obs_TEST/1.GOC/CV 1 LPR 1 (2021.08.04 13-00-29.853)_truck 0.75_2.jpg"
  ],
  'test': [
    "C:/Users/Workstation/Dropbox/DATA/_vapor_data/__tests/LPLR/test_lpr/25mai/bd225a82-54e7-4bfa-8776-636a519cf907_O_0.jpg",
    "C:/Users/Workstation/Dropbox/DATA/_vapor_data/__tests/LPLR/test_lpr/TEST_masini_grele/dataset_builder_3__DATASET_BUILDER_01__DATASET_BUILDER_5470.jpg"
  ]
}


def test(model, img_ds, prefix=''):
  for ds_name, ds in img_ds.items():
    test_ds(model=model, img_paths=ds, prefix=f'{prefix}_{ds_name}')
  # endfor
  return


def test_ds(model, img_paths, prefix=''):
  for img_path in img_paths:
    print(f'img_path: {img_path}')
    with th.no_grad():
      results = model(img_path)[0]
    np_results = results.boxes.data.cpu().numpy()
    img = cv2.imread(img_path)
    objects = []
    for np_result in np_results:
      l, t, r, b = np_result[:4]
      conf, cls = np_result[4:]
      objects.append((conf, cls))
      img = cv2.rectangle(img, (int(l), int(t)), (int(r), int(b)), (0, 255, 0), 2)
    # endfor
    print(f'Found: {objects} @{prefix}_{os.path.basename(img_path)}')
  # endfor
  return

In [ ]:
m1 = YOLO("yolov8n.yaml")
m2 = YOLO("yolov8n.yaml")
m3 = deepcopy(m1)
m4 = m1.load(trained_model_paths[0])


In [ ]:
compare_models(m1, m2)

In [ ]:
compare_models(m1, m3)

In [ ]:
compare_models(m1, m4)

In [ ]:
m5 = deepcopy(m3)
compare_models(m3, m5)

In [ ]:
m6 = m4.load(trained_model_paths[0])
compare_models(m4, m6)

In [ ]:
compare_models(m1, m6)

In [ ]:
m7 = YOLO("yolov8n.yaml").load("yolov8n.pt")

In [ ]:
models = [m1, m2, m3, m4, m5, m6, m7]
sample_images = {
    'sample': test_images['test'][:1]
}
for it in range(len(models)):
    test(models[it], sample_images, prefix=str(it + 1))

In [ ]:
### DEBUGGING TRAIN ###
# init
model = YOLO("yolov8n.yaml").load("yolov8n.pt")
m8 = deepcopy(model)
data_yaml = "lpd_no_diff"
imgsz = [448, 640]
device = 'cuda:0'

# test pre
# this 2 should be the same
test(m8, sample_images, prefix='8')
test(model, sample_images, prefix='pre')

# train
model.train(data=f'{data_yaml}.yaml', epochs=2, imgsz=imgsz, device=device)

# load best
m9 = m8.load(model.trainer.best).to(device)
model = model.load(model.trainer.best).to(device)

# test post
test(m9, test_images, prefix='9')
test(model, test_images, prefix='post')

In [ ]:
# test post
test(m8, sample_images, prefix='8')
test(m9, sample_images, prefix='9')
test(model, sample_images, prefix='post')

In [ ]:
m10 = YOLO("yolov8n.yaml").load("yolov8n.pt")
test(m10, sample_images, prefix='initial')
test(model, sample_images, prefix='post_train_best')
m11 = YOLO("yolov8n.yaml").load(model.trainer.best).to(device)
test(m11, sample_images, prefix='random_from_relative_path')
m12 = YOLO("yolov8n.yaml").load('yolov8n.pt').load(model.trainer.best).to(device)
test(m12, sample_images, prefix='pretrain_from_relative_path')

In [ ]:
best_absolute_path = 'C:/repos/edge-node/core/xperimental/lpd/runs/detect/train18/weights/best.pt'
m11 = YOLO("yolov8n.yaml").load(best_absolute_path).to(device)
test(m11, sample_images, prefix='random_from_abs_path')
m12 = YOLO("yolov8n.yaml").load('yolov8n.pt').load(best_absolute_path).to(device)
test(m12, sample_images, prefix='pretrain_from_abs_path')

In [ ]:
# loading previous train after a new one
m13 = YOLO("yolov8n.yaml").load('yolov8n.pt')
m13.train(data=f'{data_yaml}.yaml', epochs=1, imgsz=imgsz, device=device)
# test post train
test(m13, sample_images, prefix='13')
for model_path in trained_model_paths:
  curr_model = m13.load(model_path).to(device)
  test(curr_model, sample_images, prefix=os.path.basename(os.path.split(os.path.split(model_path)[0])[0]))

In [ ]:
curr_model.__dir__()

In [ ]:
model_path = 'C:/repos/edge-node/core/xperimental/lpd/runs/detect/train11/weights/best.pt'
model11 = curr_model.load(model_path).to(device)
model11.export(format="torchscript", imgsz=640, data='lpd')

In [ ]:
curr_model.export(format="torchscript", imgsz=[448, 640], data='lpd_no_diff')